##LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import boxcox
from scipy.special import boxcox1p
from termcolor import colored
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
#from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix, classification_report, RocCurveDisplay

In [ ]:
cardio=pd.read_csv("/content/cardio_train.csv")

In [ ]:
split_data = []

# Iterate over each line in the data
for line in cardio:
    # Split the line by semicolon and strip any whitespace
    split_line = [elem.strip() for elem in line.split(';')]
    # Add the split line to our data list
    split_data.append(split_line)

# Convert the list of data into a DataFrame
# We are skipping the first row because it contains the column names
data_df_manual1 = pd.DataFrame(split_data[1:], columns=split_data[0])

# Display the first few entries to verify the DataFrame structure
data_df_manual1.head()
split_data = []
expected_num_columns = None

# Iterate over each line in the data
for index, line in enumerate(cardio['id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio']):
    # Split the line by semicolon and strip any whitespace
    split_line = [elem.strip() for elem in line.split(';')]

    # Set expected number of columns based on the first line
    if index == 0:
        expected_num_columns = len(split_line)

    # Check if the current line has the expected number of columns
    if len(split_line) == expected_num_columns:
        split_data.append(split_line)
    else:
        print(f"Data format error in line {index}: {line}")

In [ ]:
cardio = pd.DataFrame(split_data, columns=data_df_manual1.columns)

In [ ]:
cardio.drop(['id'],inplace=True,axis=1)

In [ ]:
cardio['age'] = np.floor(cardio['age'].astype(float) / 365.25).astype(int)

In [ ]:
cardio['cardio'].value_counts()

cardio
0    35021
1    34979
Name: count, dtype: int64

In [ ]:
cardio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          70000 non-null  int64 
 1   gender       70000 non-null  object
 2   height       70000 non-null  object
 3   weight       70000 non-null  object
 4   ap_hi        70000 non-null  object
 5   ap_lo        70000 non-null  object
 6   cholesterol  70000 non-null  object
 7   gluc         70000 non-null  object
 8   smoke        70000 non-null  object
 9   alco         70000 non-null  object
 10  active       70000 non-null  object
 11  cardio       70000 non-null  object
dtypes: int64(1), object(11)
memory usage: 6.4+ MB


In [ ]:
continuous_features=['age','height','weight','ap_hi','ap_lo']
features_to_convert = [feature for feature in cardio.columns if feature in continuous_features]
cardio[features_to_convert] = cardio[features_to_convert].astype('float').astype('int')

In [ ]:
cardio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          70000 non-null  int64 
 1   gender       70000 non-null  object
 2   height       70000 non-null  int64 
 3   weight       70000 non-null  int64 
 4   ap_hi        70000 non-null  int64 
 5   ap_lo        70000 non-null  int64 
 6   cholesterol  70000 non-null  object
 7   gluc         70000 non-null  object
 8   smoke        70000 non-null  object
 9   alco         70000 non-null  object
 10  active       70000 non-null  object
 11  cardio       70000 non-null  object
dtypes: int64(5), object(7)
memory usage: 6.4+ MB


##DATA PREPROCESSING


In [ ]:
Q1 = cardio[continuous_features].quantile(0.25)
Q3 = cardio[continuous_features].quantile(0.75)
IQR = Q3 - Q1
outliers_count_specified = ((cardio[continuous_features] < (Q1 - 1.5 * IQR)) | (cardio[continuous_features] > (Q3 + 1.5 * IQR))).sum()

outliers_count_specified

age          4
height     519
weight    1819
ap_hi     1435
ap_lo     4632
dtype: int64

###dealing with outliers

In [ ]:
cardio[cardio["ap_hi"]<0]
cardio["ap_hi"]=cardio["ap_hi"].abs()
cardio[cardio["ap_lo"]<0]
cardio["ap_lo"]=cardio["ap_lo"].abs()

In [ ]:
def cap_and_replace_with_median(df, column):
    # Determine the 1st and 99th percentiles
    lower_bound = df[column].quantile(0.25)
    upper_bound = df[column].quantile(0.75)

    # Cap the values
    df[column] = np.clip(df[column], lower_bound, upper_bound)

    # Replace capped values with median
    median_value = df[column].median()
    df[column].where(df[column] != lower_bound, median_value, inplace=True)
    df[column].where(df[column] != upper_bound, median_value, inplace=True)

    return df

In [ ]:
#cardio=cap_and_replace_with_median(cardio, 'ap_lo')#2

In [ ]:
#cardio=cap_and_replace_with_median(cardio, 'ap_hi')

In [ ]:
#cardio=cap_and_replace_with_median(cardio, 'weight')#3

In [ ]:
#cardio=cap_and_replace_with_median(cardio, 'height')

In [ ]:
'''numeric_cols = cardio.select_dtypes(include=[np.number]).columns.tolist()

# Calculate IQR for numeric columns only
Q1 = cardio[numeric_cols].quantile(0.25)
Q3 = cardio[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as before but only for numeric columns
outliers = ((cardio[numeric_cols] < (Q1 - 1.5 * IQR)) | (cardio[numeric_cols] > (Q3 + 1.5 * IQR)))

# Drop outliers from the DataFrame
cardio = cardio[~((outliers).any(axis=1))]'''

'numeric_cols = cardio.select_dtypes(include=[np.number]).columns.tolist()\n\n# Calculate IQR for numeric columns only\nQ1 = cardio[numeric_cols].quantile(0.25)\nQ3 = cardio[numeric_cols].quantile(0.75)\nIQR = Q3 - Q1\n\n# Define outliers as before but only for numeric columns\noutliers = ((cardio[numeric_cols] < (Q1 - 1.5 * IQR)) | (cardio[numeric_cols] > (Q3 + 1.5 * IQR)))\n\n# Drop outliers from the DataFrame\ncardio = cardio[~((outliers).any(axis=1))]'

In [ ]:
cardio

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,50,2,168,62,110,80,1,1,0,0,1,0
1,55,1,156,85,140,90,3,1,0,0,1,1
2,51,1,165,64,130,70,3,1,0,0,0,1
3,48,2,169,82,150,100,1,1,0,0,1,1
4,47,1,156,56,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52,2,168,76,120,80,1,1,1,0,1,0
69996,61,1,158,126,140,90,2,2,0,0,1,1
69997,52,2,183,105,180,90,3,1,0,1,0,1
69998,61,1,163,72,135,80,1,2,0,0,0,1


In [ ]:
cardio

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,50,2,168,62,110,80,1,1,0,0,1,0
1,55,1,156,85,140,90,3,1,0,0,1,1
2,51,1,165,64,130,70,3,1,0,0,0,1
3,48,2,169,82,150,100,1,1,0,0,1,1
4,47,1,156,56,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52,2,168,76,120,80,1,1,1,0,1,0
69996,61,1,158,126,140,90,2,2,0,0,1,1
69997,52,2,183,105,180,90,3,1,0,1,0,1
69998,61,1,163,72,135,80,1,2,0,0,0,1


###Features Engineering

In [ ]:
def categorize_age(df, age_column, bins, labels):
  if age_column not in df.columns:
        raise ValueError(f"The column '{age_column}' does not exist in the DataFrame.")
  df['age_category'] = pd.cut(df[age_column], bins=bins, labels=labels,right=False, include_lowest=True)

  return df

In [ ]:
bins = [18, 30, 40, 50, 60, 101]
labels = ['18-29', '30-39', '40-49', '50-59', '60-100']
cardio = categorize_age(cardio, 'age', bins, labels)

In [ ]:
def add_bmi_category(df, height_column, weight_column):
  df['BMI'] = df[weight_column] / (df[height_column] / 100) ** 2
  conditions = [
        (df['BMI'] < 18.5),
        (df['BMI'] >= 18.5) & (df['BMI'] < 25),
        (df['BMI'] >= 25) & (df['BMI'] < 30),
        (df['BMI'] >= 30)
    ]
  categories = ['Underweight', 'Normal', 'Overweight', 'Obese']
  df['BMI_Category'] = np.select(conditions, categories, default='Unknown')

  return df

In [ ]:
cardio = add_bmi_category(cardio, 'height', 'weight')

In [ ]:
def categorize_blood_pressure(df, sys_column, dia_column):
  conditions = [
        (df[sys_column] < 120) & (df[dia_column] < 80),
        (df[sys_column] >= 120) & (df[sys_column] < 130) & (df[dia_column] < 80),
        ((df[sys_column] >= 130) & (df[sys_column] < 140)) | ((df[dia_column] >= 80) & (df[dia_column] < 89)),
        (df[sys_column] >= 140) | (df[dia_column] >= 90),
        (df[sys_column] > 180) | (df[dia_column] > 120)
    ]

  categories = [
        'Normal',
        'Elevated',
        'High Blood Pressure (Hypertension) Stage 1',
        'High Blood Pressure (Hypertension) Stage 2',
        'Hypertensive Crisis'
    ]

  df['BP_Category'] = np.select(conditions, categories, default='Unknown')

  return df

In [ ]:
cardio = categorize_blood_pressure(cardio, 'ap_hi', 'ap_lo')

In [ ]:
cardio['BP_Category'].value_counts()

BP_Category
High Blood Pressure (Hypertension) Stage 1    39850
High Blood Pressure (Hypertension) Stage 2    17397
Normal                                         9608
Elevated                                       3125
Unknown                                          20
Name: count, dtype: int64

In [ ]:
cardio

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_category,BMI,BMI_Category,BP_Category
0,50,2,168,62,110,80,1,1,0,0,1,0,50-59,21.967120,Normal,High Blood Pressure (Hypertension) Stage 1
1,55,1,156,85,140,90,3,1,0,0,1,1,50-59,34.927679,Obese,High Blood Pressure (Hypertension) Stage 2
2,51,1,165,64,130,70,3,1,0,0,0,1,50-59,23.507805,Normal,High Blood Pressure (Hypertension) Stage 1
3,48,2,169,82,150,100,1,1,0,0,1,1,40-49,28.710479,Overweight,High Blood Pressure (Hypertension) Stage 2
4,47,1,156,56,100,60,1,1,0,0,0,0,40-49,23.011177,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52,2,168,76,120,80,1,1,1,0,1,0,50-59,26.927438,Overweight,High Blood Pressure (Hypertension) Stage 1
69996,61,1,158,126,140,90,2,2,0,0,1,1,60-100,50.472681,Obese,High Blood Pressure (Hypertension) Stage 2
69997,52,2,183,105,180,90,3,1,0,1,0,1,50-59,31.353579,Obese,High Blood Pressure (Hypertension) Stage 2
69998,61,1,163,72,135,80,1,2,0,0,0,1,60-100,27.099251,Overweight,High Blood Pressure (Hypertension) Stage 1


###Encoding

Based on the feature descriptions, let's decide on one-hot encoding:

Nominal Variables: These are variables with no inherent order. They should be one-hot encoded because using them as numbers might introduce an unintended order to the model.

Ordinal Variables: These variables have an inherent order. They don't necessarily need to be one-hot encoded since their order can provide meaningful information to the model.

In [ ]:
cardio.dropna(inplace=True)

In [ ]:
#cardio = pd.get_dummies(cardio, columns=['gender', 'cardio', 'alco','smoke'], drop_first=True)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_columns = ['gender','alco','smoke','active']
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(cardio[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

cardio = pd.concat([cardio, one_hot_df], axis=1)
cardio = cardio.drop(categorical_columns, axis=1)


In [ ]:
cardio

,age,height,weight,ap_hi,ap_lo,cholesterol,gluc,cardio,age_category,BMI,BMI_Category,BP_Category,gender_1,gender_2,alco_0,alco_1,smoke_0,smoke_1,active_0,active_1
0,50,168,62,110,80,1,1,0,50-59,21.967120,Normal,High Blood Pressure (Hypertension) Stage 1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
1,55,156,85,140,90,3,1,1,50-59,34.927679,Obese,High Blood Pressure (Hypertension) Stage 2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,51,165,64,130,70,3,1,1,50-59,23.507805,Normal,High Blood Pressure (Hypertension) Stage 1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,48,169,82,150,100,1,1,1,40-49,28.710479,Overweight,High Blood Pressure (Hypertension) Stage 2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,47,156,56,100,60,1,1,0,40-49,23.011177,Normal,Normal,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52,168,76,120,80,1,1,0,50-59,26.927438,Overweight,High Blood Pressure (Hypertension) Stage 1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
69996,61,158,126,140,90,2,2,1,60-100,50.472681,Obese,High Blood Pressure (Hypertension) Stage 2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
69997,52,183,105,180,90,3,1,1,50-59,31.353579,Obese,High Blood Pressure (Hypertension) Stage 2,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
69998,61,163,72,135,80,1,2,1,60-100,27.099251,Overweight,High Blood Pressure (Hypertension) Stage 1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
def custom_label_encoder(df, column, order):
    mapping = {category: idx for idx, category in enumerate(order)}
    df[column + ' Code'] = df[column].map(mapping)

    return df

In [ ]:
cardio.age_category.unique()

['50-59', '40-49', '60-100', '30-39', '18-29']
Categories (5, object): ['18-29' < '30-39' < '40-49' < '50-59' < '60-100']

In [ ]:
order=['Normal', 'Obese', 'Overweight', 'Underweight']

In [ ]:
cardio = custom_label_encoder(cardio, 'BMI_Category', order)

In [ ]:
order=['Unknown','Normal', 'Elevated', 'High Blood Pressure (Hypertension) Stage 1', 'High Blood Pressure (Hypertension) Stage 2']

In [ ]:
cardio = custom_label_encoder(cardio, 'BP_Category', order)

In [ ]:
order=['18-29', '30-39' ,'40-49' , '50-59' , '60-100']

In [ ]:
cardio = custom_label_encoder(cardio, 'age_category', order)

In [ ]:
cardio.drop(['BMI_Category','BP_Category','BMI','age_category'],axis=1,inplace=True)

In [ ]:
cardio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   age                70000 non-null  int64   
 1   height             70000 non-null  int64   
 2   weight             70000 non-null  int64   
 3   ap_hi              70000 non-null  int64   
 4   ap_lo              70000 non-null  int64   
 5   cholesterol        70000 non-null  object  
 6   gluc               70000 non-null  object  
 7   cardio             70000 non-null  object  
 8   gender_1           70000 non-null  float64 
 9   gender_2           70000 non-null  float64 
 10  alco_0             70000 non-null  float64 
 11  alco_1             70000 non-null  float64 
 12  smoke_0            70000 non-null  float64 
 13  smoke_1            70000 non-null  float64 
 14  active_0           70000 non-null  float64 
 15  active_1           70000 non-null  float64 
 16  BMI_

In [ ]:
cardio.dropna(inplace=True)

In [ ]:
features_to_convert=['cholesterol','gluc','cardio','age_category Code']

In [ ]:
cardio[features_to_convert] = cardio[features_to_convert].astype('int')

In [ ]:
cardio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                70000 non-null  int64  
 1   height             70000 non-null  int64  
 2   weight             70000 non-null  int64  
 3   ap_hi              70000 non-null  int64  
 4   ap_lo              70000 non-null  int64  
 5   cholesterol        70000 non-null  int64  
 6   gluc               70000 non-null  int64  
 7   cardio             70000 non-null  int64  
 8   gender_1           70000 non-null  float64
 9   gender_2           70000 non-null  float64
 10  alco_0             70000 non-null  float64
 11  alco_1             70000 non-null  float64
 12  smoke_0            70000 non-null  float64
 13  smoke_1            70000 non-null  float64
 14  active_0           70000 non-null  float64
 15  active_1           70000 non-null  float64
 16  BMI_Category Code  700

In [ ]:
cardio

,age,height,weight,ap_hi,ap_lo,cholesterol,gluc,cardio,gender_1,gender_2,alco_0,alco_1,smoke_0,smoke_1,active_0,active_1,BMI_Category Code,BP_Category Code,age_category Code
0,50,168,62,110,80,1,1,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0,3,3
1,55,156,85,140,90,3,1,1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1,4,3
2,51,165,64,130,70,3,1,1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,3,3
3,48,169,82,150,100,1,1,1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2,4,2
4,47,156,56,100,60,1,1,0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52,168,76,120,80,1,1,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2,3,3
69996,61,158,126,140,90,2,2,1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1,4,4
69997,52,183,105,180,90,3,1,1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1,4,3
69998,61,163,72,135,80,1,2,1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2,3,4


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
columns_to_scale = ['age', 'height', 'weight','ap_lo','ap_hi']

scaler = StandardScaler()

cardio[columns_to_scale] = scaler.fit_transform(cardio[columns_to_scale])

In [ ]:
cardio

,age,height,weight,ap_hi,ap_lo,cholesterol,gluc,cardio,gender_1,gender_2,alco_0,alco_1,smoke_0,smoke_1,active_0,active_1,BMI_Category Code,BP_Category Code,age_category Code
0,-0.414535,0.443452,-0.847767,-0.122355,-0.088250,1,1,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0,3,3
1,0.324846,-1.018168,0.749916,0.072463,-0.035191,3,1,1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1,4,3
2,-0.266659,0.078047,-0.708838,0.007524,-0.141308,3,1,1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,3,3
3,-0.710287,0.565254,0.541522,0.137402,0.017868,1,1,1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2,4,2
4,-0.858164,-1.018168,-1.264554,-0.187294,-0.194367,1,1,0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,-0.118783,0.443452,0.124735,-0.057416,-0.088250,1,1,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2,3,3
69996,1.212103,-0.774565,3.597959,0.072463,-0.035191,2,2,1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1,4,4
69997,-0.118783,2.270477,2.139205,0.332220,-0.035191,3,1,1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1,4,3
69998,1.212103,-0.165556,-0.153123,0.039993,-0.088250,1,2,1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2,3,4


In [ ]:
#cardio=cardio.drop_duplicates(keep='last')

##DATA SPLITING AND MODELING


In [ ]:
x = cardio.drop('cardio', axis=1)
y = cardio['cardio']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

###gradientboosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
# Hyperparameter distribution
param_dist = {
    'n_estimators': [100, 200, 300, 400],  # number of boosting stages
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # shrinks the contribution of each tree
    'max_depth': [3, 5, 8, 10],  # maximum depth of the individual regression estimators
    'min_samples_split': [2, 4, 6],  # minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # minimum number of samples required to be at a leaf node
    'subsample': [0.6, 0.8, 1.0],  # fraction of samples to be used for fitting the individual base learners
    'max_features': ['sqrt', 'log2', None]  # number of features to consider when looking for the best split
}

# Initialize GradientBoosting Classifier
gbc = GradientBoostingClassifier(random_state=42)

# Initialize RandomizedSearchCV
gbc_random_search = RandomizedSearchCV(estimator=gbc, param_distributions=param_dist,
                                       n_iter=50, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit RandomizedSearchCV to the data
gbc_random_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = gbc_random_search.best_params_
best_gbc = gbc_random_search.best_estimator_

print(f'Best parameters: {best_params}')

# Predict the labels of the test set with the best estimator
y_pred = best_gbc.predict(X_test)

# Evaluate the best estimator
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f'Accuracy of best estimator: {accuracy:.4f}')
print(f'Recall of best estimator: {recall:.4f}')
print(f'Confusion Matrix of best estimator:\n{cm}')


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters: {'subsample': 0.8, 'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 8, 'learning_rate': 0.01}
Accuracy of best estimator: 0.7372
Recall of best estimator: 0.6944
Confusion Matrix of best estimator:
[[5452 1536]
 [2143 4869]]


###lightgbm

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

# Hyperparameter distribution
param_dist = {
    'num_leaves': [31, 50, 70, 100],
    'boosting_type': ['gbdt', 'dart'],
    'max_depth': [-1, 5, 10, 20],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'subsample_for_bin': [20000, 50000, 100000],
    'min_split_gain': [0.0, 0.1, 0.2],
    'min_child_weight': [0.001, 0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 1.0],
    'reg_lambda': [0.0, 0.1, 1.0]
}

# Initialize LGBM Classifier
lgbm = lgb.LGBMClassifier(random_state=42)

# Initialize RandomizedSearchCV
lgbm_random_search = RandomizedSearchCV(estimator=lgbm, param_distributions=param_dist,
                                        n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit RandomizedSearchCV to the data
lgbm_random_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = lgbm_random_search.best_params_
best_lgbm = lgbm_random_search.best_estimator_

print(f'Best parameters: {best_params}')

# Predict the labels of the test set with the best estimator
y_pred = best_lgbm.predict(X_test)

# Evaluate the best estimator
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f'Accuracy of best estimator: {accuracy:.4f}')
print(f'Recall of best estimator: {recall:.4f}')
print(f'Confusion Matrix of best estimator:\n{cm}')


Fitting 3 folds for each of 100 candidates, totalling 300 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 27967, number of negative: 28033
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012524 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 487
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499411 -> initscore=-0.002357
[LightGBM] [Info] Start training from score -0.002357
Best parameters: {'subsample_for_bin': 100000, 'subsample': 0.8, 'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n

###Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:

input_shape = X_train.shape[1]  # Number of features

model = Sequential([
    Dense(256, activation='relu', input_shape=(input_shape,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=70, validation_split=0.2, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

Epoch 1/70
1400/1400 [==============================] - 6s 3ms/step - loss: 0.5752 - accuracy: 0.7035 - val_loss: 0.5516 - val_accuracy: 0.7263
Epoch 2/70
1400/1400 [==============================] - 4s 3ms/step - loss: 0.5539 - accuracy: 0.7233 - val_loss: 0.5447 - val_accuracy: 0.7298
Epoch 3/70
1400/1400 [==============================] - 5s 4ms/step - loss: 0.5497 - accuracy: 0.7272 - val_loss: 0.5470 - val_accuracy: 0.7310
Epoch 4/70
1400/1400 [==============================] - 5s 4ms/step - loss: 0.5480 - accuracy: 0.7283 - val_loss: 0.5489 - val_accuracy: 0.7269
Epoch 5/70
1400/1400 [==============================] - 8s 6ms/step - loss: 0.5458 - accuracy: 0.7298 - val_loss: 0.5450 - val_accuracy: 0.7331
Epoch 6/70
1400/1400 [==============================] - 4s 3ms/step - loss: 0.5448 - accuracy: 0.7304 - val_loss: 0.5430 - val_accuracy: 0.7369
Epoch 7/70
1400/1400 [==============================] - 4s 3ms/step - loss: 0.5434 - accuracy: 0.7311 - val_loss: 0.5401 - val_accuracy:

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Predict the labels (probabilities)
predictions_prob = model.predict(X_test)

# Convert probabilities to binary predictions
predictions_binary = (predictions_prob >= 0.5).astype(int)

# Flatten the predictions array to match the shape of y_test
predictions_binary = predictions_binary.flatten()

# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, predictions_binary)
print(f"Accuracy: {accuracy_sklearn:.4f}")

# Calculate recall
recall = recall_score(y_test, predictions_binary)
print(f"Recall: {recall:.4f}")

# Calculate and print confusion matrix
conf_matrix = confusion_matrix(y_test, predictions_binary)
print("Confusion Matrix:")
print(conf_matrix)

438/438 [==============================] - 1s 3ms/step - loss: 0.6363 - accuracy: 0.7183
Test Loss: 0.6363, Test Accuracy: 0.7183
438/438 [==============================] - 2s 3ms/step
Accuracy: 0.7183
Recall: 0.6433
Confusion Matrix:
[[5545 1443]
 [2501 4511]]


###DT

In [ ]:
dt_base = DecisionTreeClassifier(random_state=0)

In [ ]:
def tune_clf_hyperparameters(clf, param_grid, X_train, y_train, scoring='recall', n_splits=3):
    # Create the cross-validation object using StratifiedKFold to ensure the class distribution is the same across all the folds
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

    # Create the GridSearchCV object
    clf_grid = GridSearchCV(clf, param_grid, cv=cv, scoring=scoring, n_jobs=-1)

    # Fit the GridSearchCV object to the training data
    clf_grid.fit(X_train, y_train)

    # Get the best hyperparameters
    best_hyperparameters = clf_grid.best_params_

    # Return best_estimator_ attribute which gives us the best model that has been fitted to the training data
    return clf_grid.best_estimator_, best_hyperparameters

In [ ]:
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2,3],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2]
}

In [ ]:
best_dt, best_dt_hyperparams = tune_clf_hyperparameters(dt_base, param_grid_dt, X_train, y_train)

In [ ]:
print('DT Optimal Hyperparameters: \n', best_dt_hyperparams)

DT Optimal Hyperparameters: 
 {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [ ]:
print(classification_report(y_train, best_dt.predict(X_train)))

              precision    recall  f1-score   support

           0       0.68      0.77      0.72     25775
           1       0.75      0.67      0.71     27273

    accuracy                           0.71     53048
   macro avg       0.72      0.72      0.71     53048
weighted avg       0.72      0.71      0.71     53048



In [ ]:
print(classification_report(y_test, best_dt.predict(X_test)))

              precision    recall  f1-score   support

           0       0.68      0.75      0.71      6444
           1       0.74      0.66      0.70      6818

    accuracy                           0.70     13262
   macro avg       0.71      0.71      0.70     13262
weighted avg       0.71      0.70      0.70     13262



In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    """
    Evaluates the performance of a trained model on test data using various metrics.
    """
    # Make predictions
    y_pred = model.predict(X_test)

    # Get classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Extracting metrics
    metrics = {
        "precision_0": report["0"]["precision"],
        "precision_1": report["1"]["precision"],
        "recall_0": report["0"]["recall"],
        "recall_1": report["1"]["recall"],
        "f1_0": report["0"]["f1-score"],
        "f1_1": report["1"]["f1-score"],
        "macro_avg_precision": report["macro avg"]["precision"],
        "macro_avg_recall": report["macro avg"]["recall"],
        "macro_avg_f1": report["macro avg"]["f1-score"],
        "accuracy": accuracy_score(y_test, y_pred)
    }

    # Convert dictionary to dataframe
    df = pd.DataFrame(metrics, index=[model_name]).round(2)

    return df

In [ ]:
dt_evaluation = evaluate_model(best_dt, X_test, y_test, 'DT')
dt_evaluation

,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1,macro_avg_precision,macro_avg_recall,macro_avg_f1,accuracy
DT,0.68,0.74,0.75,0.66,0.71,0.7,0.71,0.71,0.7,0.7


###Random forest

In [ ]:
rf_base = RandomForest
Classifier(random_state=0)

NameError: name 'RandomForest' is not defined

In [ ]:
param_grid_rf = {
    'n_estimators': [10, 30, 50, 70, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3],
    'bootstrap': [True, False]
}

In [ ]:
best_rf, best_rf_hyperparams = tune_clf_hyperparameters(rf_base, param_grid_rf, X_train, y_train)
print('RF Optimal Hyperparameters: \n', best_rf_hyperparams)

RF Optimal Hyperparameters: 
 {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
print(classification_report(y_train, best_rf.predict(X_train)))

              precision    recall  f1-score   support

           0       0.69      0.76      0.72     25775
           1       0.75      0.67      0.71     27273

    accuracy                           0.72     53048
   macro avg       0.72      0.72      0.72     53048
weighted avg       0.72      0.72      0.72     53048



In [ ]:
print(classification_report(y_test, best_rf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.68      0.75      0.72      6444
           1       0.74      0.67      0.70      6818

    accuracy                           0.71     13262
   macro avg       0.71      0.71      0.71     13262
weighted avg       0.71      0.71      0.71     13262



In [ ]:
rf_evaluation = evaluate_model(best_rf, X_test, y_test, 'RF')
rf_evaluation

,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1,macro_avg_precision,macro_avg_recall,macro_avg_f1,accuracy
RF,0.68,0.74,0.75,0.67,0.72,0.7,0.71,0.71,0.71,0.71


###KNN

In [ ]:
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

In [ ]:
knn_param_grid = {
    'knn__n_neighbors': list(range(1, 12)),
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]  # 1: Manhattan distance, 2: Euclidean distance
}

In [ ]:
best_knn, best_knn_hyperparams = tune_clf_hyperparameters(knn_pipeline, knn_param_grid, X_train, y_train)
print('KNN Optimal Hyperparameters: \n', best_knn_hyperparams)

KNN Optimal Hyperparameters: 
 {'knn__n_neighbors': 11, 'knn__p': 2, 'knn__weights': 'uniform'}


In [ ]:
print(classification_report(y_train, best_knn.predict(X_train)))

              precision    recall  f1-score   support

           0       0.73      0.76      0.75     25775
           1       0.77      0.74      0.75     27273

    accuracy                           0.75     53048
   macro avg       0.75      0.75      0.75     53048
weighted avg       0.75      0.75      0.75     53048



In [ ]:
print(classification_report(y_test, best_knn.predict(X_test)))

              precision    recall  f1-score   support

           0       0.68      0.71      0.70      6444
           1       0.72      0.69      0.70      6818

    accuracy                           0.70     13262
   macro avg       0.70      0.70      0.70     13262
weighted avg       0.70      0.70      0.70     13262



In [ ]:
knn_evaluation = evaluate_model(best_knn, X_test, y_test, 'KNN')
knn_evaluation

,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1,macro_avg_precision,macro_avg_recall,macro_avg_f1,accuracy
KNN,0.68,0.72,0.71,0.69,0.7,0.7,0.7,0.7,0.7,0.7


###SVM

In [ ]:
columns_to_scale = ['age', 'height', 'weight','ap_lo','ap_hi']

scaler = StandardScaler()

cardio[columns_to_scale] = scaler.fit_transform(cardio[columns_to_scale])


In [ ]:
param_grid_svm = {
    'svm__C': [0.0011, 0.005, 0.01, 0.05, 0.1, 1, 10, 20],
    'svm__kernel': ['linear', 'rbf', 'poly'],
    'svm__gamma': ['scale', 'auto', 0.1, 0.5, 1, 5],
    'svm__degree': [2, 3, 4]
}

In [ ]:
svm_pipeline = Pipeline([
    ('svm', SVC(probability=True))
])

In [ ]:
best_svm, best_svm_hyperparams = tune_clf_hyperparameters(svm_pipeline, param_grid_svm, X_train, y_train)
print('SVM Optimal Hyperparameters: \n', best_svm_hyperparams)

NameError: name 'tune_clf_hyperparameters' is not defined

In [ ]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.8 MB

In [ ]:
from pycaret.classification import *
s = setup(cardio,target='cardio', train_size = 0.8, fold = 5)

,Description,Value
0,Session id,2906
1,Target,cardio
2,Target type,Binary
3,Original data shape,"(70000, 19)"
4,Transformed data shape,"(70000, 19)"
5,Transformed train set shape,"(56000, 19)"
6,Transformed test set shape,"(14000, 19)"
7,Numeric features,18
8,Preprocess,True
9,Imputation type,simple


In [ ]:
cardio

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI_Category Code,BP_Category Code,age_category Code
0,-0.417718,2,0.488772,-0.949151,-1.124463,-0.148339,1,1,0,0,1,0,0,3,3
1,0.317906,1,-1.161112,0.831978,0.876183,0.406631,3,1,0,0,1,1,1,4,3
2,-0.270593,1,0.076301,-0.794270,0.209301,-0.703308,3,1,0,0,0,1,0,3,3
3,-0.711968,2,0.626262,0.599657,1.543065,0.961600,1,1,0,0,1,1,2,4,2
4,-0.859092,1,-1.161112,-1.413793,-1.791345,-0.148339,1,1,0,0,0,0,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,-0.123469,2,0.488772,0.135014,-0.457581,-0.148339,1,1,1,0,1,0,2,3,3
69996,1.200655,1,-0.886131,-0.174747,0.876183,0.406631,2,2,0,0,1,1,2,4,4
69997,-0.123469,2,2.551127,2.380786,-0.457581,0.406631,3,1,0,1,0,1,1,4,3
69998,1.200655,1,-0.198680,-0.174747,0.542742,-0.148339,1,2,0,0,0,1,2,3,4


In [ ]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7361,0.8029,0.6996,0.7545,0.7260,0.4722,0.4735,1.9680
gbc,Gradient Boosting Classifier,0.7354,0.8031,0.7014,0.7522,0.7259,0.4707,0.4718,4.5220
xgboost,Extreme Gradient Boosting,0.7336,0.7973,0.6967,0.7519,0.7232,0.4671,0.4684,1.0280
ada,Ada Boost Classifier,0.7291,0.7955,0.6590,0.7662,0.7086,0.4582,0.4627,1.3540
lr,Logistic Regression,0.7200,0.7807,0.6990,0.7294,0.7139,0.4401,0.4405,2.6820
rf,Random Forest Classifier,0.7025,0.7614,0.7025,0.7023,0.7024,0.4051,0.4051,6.6040
svm,SVM - Linear Kernel,0.6958,0.7844,0.5822,0.7682,0.6425,0.3914,0.4137,0.3780
ridge,Ridge Classifier,0.6949,0.7573,0.7121,0.6882,0.7000,0.3899,0.3901,0.1060
lda,Linear Discriminant Analysis,0.6949,0.7573,0.7121,0.6882,0.7000,0.3899,0.3901,0.1340
et,Extra Trees Classifier,0.6864,0.7297,0.6890,0.6852,0.6871,0.3728,0.3728,8.0780


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_train, best_svm.predict(X_train)))

In [ ]:
print(classification_report(y_test, best_svm.predict(X_test)))

In [ ]:
svm_evaluation = evaluate_model(best_svm, X_test, y_test, 'SVM')
svm_evaluation

In [ ]:
lgbm = LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    random_state=42
)


In [ ]:
param_grid = {
    'num_leaves': [20, 30, 40],  # Reduce complexity
     'min_data_in_leaf': [10, 20, 30],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'lambda_l1': [0, 0.1, 0.5],
    'lambda_l2': [0, 0.1, 0.5],
    'min_gain_to_split': [0.0, 0.1],  # Allows splits with smaller gains
    'max_depth': [3, 5, 7],  # Slightly deeper trees
}


In [ ]:
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

In [ ]:
grid_search.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=10, 

In [ ]:
best_lgbm = lgbm_search.best_estimator_
best_lgbm_params = lgbm_search.best_params_

print('LightGBM Optimal Hyperparameters:\n', best_lgbm_params)

In [ ]:
print(classification_report(y_train, best_lgbm_params.predict(X_train)))

###XGBOOST


In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Define parameters to search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_parameters}')
print(f'Best score: {best_score}')

# Train the model with the best parameters
best_clf = grid_search.best_estimator_

# Predict the labels of the test set
y_pred = best_clf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f'Accuracy of best classifier: {accuracy}')
print(f'Recall of best classifier: {recall}')
print(f'Confusion Matrix of best classifier:\n{cm}')

Fitting 3 folds for each of 324 candidates, totalling 972 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.8}
Best score: 0.7361964038491736
Accuracy of best classifier: 0.7422142857142857
Recall of best classifier: 0.7062179121505989
Confusion Matrix of best classifier:
[[5439 1549]
 [2060 4952]]


In [ ]:
pip install joblib


In [ ]:
from joblib import dump

dump(best_clf, 'best_clf.joblib')


['best_clf.joblib']